In [1]:
from multiprocessing import Pool
import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm

In [2]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens

In [3]:
"""BCS"""
def sigmoid(x):
    try:
        return 1/(1+math.exp(-x))
    except OverflowError:
        return 0.000001
def sigma(beta):
    p=math.gamma(1+beta)* math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*(pow(2,(beta-1)/2)))
    return pow(p,1/beta)
def levy_flight(beta,best,est,alpha):
    sg=sigma(beta)
    u=np.random.normal(0,sg**2)
    v=abs(np.random.normal(0,1))
    step=u/pow(v,1/beta)
    step_size=alpha+step#+(step*(est-best))
    new=est+step_size#*np.random.normal()#random.normalvariate(0,sg)
    return new

def BCS(Eval_Func,m_i=200,n=20,minf=0,dim=None,prog=False,alpha=0.1,beta=1.5,param=0.25,mp=None):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            alpha and beta: Arguments in levy flight, default=0.1,1.5
            param: Probability to destroy inferior nest, default=0.25(25%)
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)
    pa=param
    #flag=dr
    gens=random_search(n,dim)
    fit=[float("-inf") if minf == 0 else float("inf") for _ in range(n)]
    pos=[0 for _ in range(n)]
    g_pos=[0]*dim
    g_val=float("-inf") if minf == 0 else float("inf")
    gens_dict={tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}
    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)
    for it in miter:
        if mp!=None:
            fnew=[gens_dict[tuple(g)]  if tuple(g) in gens_dict  else(float("-inf") if minf == 0 else float("inf")) for g in gens]
            alter_gens=[i for i,g in enumerate(gens) if tuple(g) not in gens_dict]
            #print(len(alter_gens))
            with Pool(mp) as p:
                alter_fit = p.map(estimate,[gens[_i] for _i in alter_gens])
            z=0
            for i in range(len(fit)):
                if i in alter_gens:
                    fnew[i]=alter_fit[z]
                    gens_dict[tuple(gens[i])]=alter_fit[z]
                    z+=1
                else:pass
                if fnew[i] > fit[i] if minf==0 else fnew[i] < fit[i]:
                    fit[i]=dc(fnew[i])
                    pos[i]=dc(gens[i])

        else:
            for i,g in enumerate(gens):
                if tuple(g) in gens_dict:
                    score=gens_dict[tuple(g)]
                else:
                    score=estimate(g)
                    gens_dict[tuple(g)]=score
                if score > fit[i] if minf==0 else score < fit[i]:
                    fit[i]=score
                    pos[i]=g

        maxfit,maxind=max(fit),fit.index(max(fit))
        minfit,minind=min(fit),fit.index(min(fit))
        if minf==0:
            if maxfit > g_val:
                g_val=dc(maxfit)
                g_pos=dc(gens[maxind])
        else:
            if minfit < g_val:
                g_val=dc(minfit)
                g_pos=dc(gens[minind])

        if pa < random.uniform(0,1):
            if minf==0:
                gens[minind]=[0 if 0.5>random.uniform(0,1) else 1 for _ in range(dim)]#rand_gen()
                fit[minind]=float("-inf") if minf == 0 else float("inf")
            else:
                gens[maxind]=[0 if 0.5>random.uniform(0,1) else 1 for _ in range(dim)]#rand_gen()
                fit[maxind]=float("-inf") if minf == 0 else float("inf")


        for g in gens:
            for d in range(dim):
                x=levy_flight(beta,g_pos[d],g[d],alpha)
                if random.uniform(0,1) < sigmoid(x):
                    g[d]=1
                else:
                    g[d]=0
    print(len(gens_dict))
    return g_val,g_pos,g_pos.count(1)

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import numpy as np

from sklearn import svm
 
SEED = 2018
np.random.seed(SEED)
df_train = pd.read_csv("VoxForge512.csv")
X = np.array(df_train.drop(['label'],axis=1))
y = np.array(df_train['label'])
from sklearn.model_selection import train_test_split
df=df_train
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
# trainX=data
# trainy=label
cross=4
test_size=(1/cross)
trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)

900 201


In [5]:
if __name__ == '__main__':
    from sklearn import svm
    from time import time

    np.random.seed(20)
    tr_d=trainX
    te_d=testX
    tr_l=trainy
    te_l=testy

In [6]:
class Evaluate:#setting class
        def __init__(self):#set train_data,label,test_data,label
            self.train_l=tr_l
            self.train_d=tr_d
            self.test_l=te_l
            self.test_d=te_d
        def evaluate(self,gen):
            """
            Setting of evaluation function.
            Here, the correct answer rate is used.
              anser_label/all_label
            """
            mask=np.array(gen) > 0
            al_data=np.array([al[mask] for al in self.train_d])
            al_test_data=np.array([al[mask] for al in self.test_d])
            #↑masking with [01]sequence list
            res=RandomForestClassifier(n_jobs = -1, verbose = 0, n_estimators=5, criterion='entropy', random_state = 0).fit(al_data,self.train_l).predict(al_test_data)
            return np.count_nonzero(self.test_l==res)/len(self.test_l)
            #↑evaluate with fittness function
        def check_dimentions(self,dim):#check number of all feature
            if dim==None:
                return len(self.train_d[0])
            else:
                return dim

In [19]:
%%time
s,g,l=BCS(Eval_Func=Evaluate, n=3, m_i=5)

16
CPU times: user 754 ms, sys: 49.3 ms, total: 803 ms
Wall time: 3.44 s


In [20]:
l

160

In [21]:
List = []

for i in range(0,len(g)):
    if g[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("VoxForge512.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,label
0,0.003289,0.003680,0.002809,0.002662,0.003032,0.002435,0.004380,0.001103,0.002665,0.002319,...,6.368087,15.037514,14.887147,17.031933,11.462402,0.016049,0.031459,0.009660,0.017622,English
1,0.002735,0.002047,0.002489,0.002068,0.001964,0.003271,0.001584,0.000306,0.002996,0.001032,...,8.133885,19.207247,15.687752,17.947880,11.421647,0.014899,0.029206,0.013936,0.024158,English
2,0.001556,0.001723,0.001227,0.001315,0.001354,0.000923,0.001206,0.000508,0.001195,0.001047,...,13.792500,32.569420,17.805973,20.371273,11.088083,0.014856,0.029121,0.015527,0.028974,English
3,0.007180,0.010772,0.011593,0.007111,0.011722,0.007152,0.008713,0.002654,0.008530,0.003775,...,4.076348,9.625833,1.745809,1.997327,9.626831,0.011871,0.023270,0.010665,0.016978,English
4,0.003632,0.004742,0.003760,0.002512,0.004911,0.004053,0.004527,0.002201,0.003138,0.001852,...,4.520422,10.674464,5.230475,5.984027,9.285885,0.011261,0.022074,0.008326,0.017641,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000754,0.000790,0.000955,0.000608,0.001097,0.000490,0.000559,0.000482,0.000726,0.000330,...,0.001004,0.002370,0.001476,0.001688,29.929813,0.032924,0.064538,0.025733,0.011313,Spanish
896,0.000093,0.000087,0.000061,0.000111,0.000120,0.000100,0.000161,0.000117,0.000092,0.000101,...,0.002488,0.005875,0.001386,0.001586,9.986907,0.005506,0.010794,0.003350,0.002254,Spanish
897,0.000122,0.000108,0.000083,0.000059,0.000099,0.000062,0.000040,0.000025,0.000042,0.000049,...,0.000358,0.000845,0.000601,0.000688,9.939436,0.006557,0.012853,0.005161,0.003221,Spanish
898,0.000005,0.000010,0.000008,0.000005,0.000007,0.000006,0.000005,0.000009,0.000008,0.000007,...,0.000236,0.000557,0.000454,0.000519,9.965343,0.004320,0.008469,0.003256,0.002046,Spanish


In [22]:
df_train1.to_csv('VF_BCS.csv')